In [3]:
from pathlib import Path

from ray.tune.result import TRAINING_ITERATION
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.core import (
    COMPONENT_LEARNER,
    COMPONENT_LEARNER_GROUP,
    COMPONENT_RL_MODULE,
)
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.examples.envs.classes.multi_agent import MultiAgentPendulum
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EPISODE_RETURN_MEAN,
    NUM_ENV_STEPS_SAMPLED_LIFETIME,
)
from ray.rllib.utils.numpy import convert_to_numpy
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    check,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4

from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env
import os

import gymnasium as gym
import numpy as np
import tree  # pip install dm_tree

from ray.rllib.core import DEFAULT_MODULE_ID
from ray.rllib.core.columns import Columns
from ray.rllib.core.rl_module.rl_module import RLModule
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.numpy import convert_to_numpy, softmax
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EPISODE_RETURN_MEAN,
)
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls

In [5]:
import sys
sys.argv = [
    'notebook_script.py',
    '--enable-new-api-stack',
    '--num-agents=2',
    # '--no-tune',
    '--checkpoint-at-end',
    '--stop-reward=200.0',

]
print("参数设置完成")

参数设置完成


In [18]:
parser = add_rllib_example_script_args(
    default_iters=2,
    default_timesteps=10000,
    default_reward=0.0,
)

args = parser.parse_args()

assert args.num_agents > 0, "Must set --num-agents > 0 when running this script!"
assert (
    args.enable_new_api_stack
), "Must set --enable-new-api-stack when running this script!"

print(f"参数解析完成: num_agents={args.num_agents}, algo={args.algo}")


参数解析完成: num_agents=2, algo=PPO


In [21]:
# Cell 4: 注册环境和配置算法
# Here, we use the "Agent Environment Cycle" (AEC) PettingZoo environment type.
# For a "Parallel" environment example, see the rock paper scissors examples
# in this same repository folder.
# Here, we use the "Agent Environment Cycle" (AEC) PettingZoo environment type.
# For a "Parallel" environment example, see the rock paper scissors examples
# in this same repository folder.
register_env("env", lambda _: PettingZooEnv(waterworld_v4.env()))

# Policies are called just like the agents (exact 1:1 mapping).
policies = {f"pursuer_{i}" for i in range(args.num_agents)}

base_config = (
    get_trainable_cls(args.algo)
    .get_default_config()
    .environment("env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .training(
        vf_loss_coeff=0.005,
    )
    .rl_module(
        rl_module_spec=MultiRLModuleSpec(
            rl_module_specs={p: RLModuleSpec() for p in policies},
        ),
        model_config=DefaultModelConfig(vf_share_layers=True),
    )
)

# run_rllib_example_script_experiment(base_config, args)
results = run_rllib_example_script_experiment(base_config, args, keep_ray_up=True)
print("环境注册和配置完成")

2025-06-24 15:13:23,307	INFO worker.py:1917 -- Started a local Ray instance.


== Status ==
Current time: 2025-06-24 15:13:23 (running for 00:00:00.12)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-13-22_768088_547330/artifacts/2025-06-24_15-13-23/PPO_2025-06-24_15-13-23/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(PPO pid=552889) 2025-06-24 15:13:25,316	WARNING algorithm_config.py:5014 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
(MultiAgentEnvRunner pid=552974) 2025-06-24 15:13:27,049	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
(PPO pid=552889) Install gputil for GPU system monitoring.


== Status ==
Current time: 2025-06-24 15:13:28 (running for 00:00:05.14)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-13-22_768088_547330/artifacts/2025-06-24_15-13-23/PPO_2025-06-24_15-13-23/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




Trial name,env_runner_group,env_runners,fault_tolerance,learners,num_env_steps_sampled_lifetime,num_training_step_calls_per_iteration,perf,timers
PPO_env_55877_00000,{'actor_manager_num_outstanding_async_reqs': 0},"{'module_to_env_connector': {'timers': {'connectors': {'remove_single_ts_time_rank_from_batch': np.float64(9.8477529672102e-07), 'tensor_to_numpy': np.float64(3.1459986354376984e-05), 'un_batch_to_individual_items': np.float64(1.3426101270532106e-05), 'normalize_and_clip_actions': np.float64(3.524928129802899e-05), 'module_to_agent_unmapping': np.float64(2.6854698092063245e-06), 'listify_data_for_vector_env': np.float64(5.991409915151327e-06), 'get_actions': np.float64(8.183217608720008e-05)}}, 'connector_pipeline_timer': np.float64(0.00022520500802222047)}, 'num_env_steps_sampled': 4000.0, 'episode_return_min': -309.4820964435785, 'timers': {'connectors': {'numpy_to_tensor': np.float64(4.1812993003986776e-05), 'add_states_from_episodes_to_batch': np.float64(4.945002729073167e-06), 'add_observations_from_episodes_to_batch': np.float64(2.724550722632557e-05), 'batch_individual_items': np.float64(2.2136999177746475e-05), 'agent_to_module_mapping': np.float64(4.642497515305877e-06), 'add_time_dim_to_batch_and_zero_pad': np.float64(1.3065495295450091e-05)}}, 'env_to_module_connector': {'timers': {'connectors': {'numpy_to_tensor': np.float64(2.2342237183981633e-05), 'add_states_from_episodes_to_batch': np.float64(4.032387564088161e-06), 'batch_individual_items': np.float64(1.2592065599674422e-05), 'add_observations_from_episodes_to_batch': np.float64(1.5832307610279334e-05), 'agent_to_module_mapping': np.float64(3.2720770236916246e-06), 'add_time_dim_to_batch_and_zero_pad': np.float64(6.898764856134054e-06)}}, 'connector_pipeline_timer': np.float64(0.00010648422869347633)}, 'num_agent_steps_sampled': {'pursuer_0': 2000.0, 'pursuer_1': 2004.0}, 'episode_len_mean': 1000.0, 'num_episodes': 4.0, 'episode_len_max': 1000, 'num_module_steps_sampled': {'pursuer_1': 2004.0, 'pursuer_0': 2000.0}, 'num_agent_steps_sampled_lifetime': {'pursuer_0': 4000.0, 'pursuer_1': 4008.0}, 'env_reset_timer': np.float64(0.0015643125007045455), 'env_to_module_sum_episodes_length_in': np.float64(891.0423566894266), 'module_episode_returns_mean': {'pursuer_1': -200.61821661608354, 'pursuer_0': -80.28831977246807}, 'num_env_steps_sampled_lifetime': 8000.0, 'episode_duration_sec_mean': 0.9735575072536449, 'sample': np.float64(1.9717855798754318), 'num_episodes_lifetime': 8.0, 'env_step_timer': np.float64(0.00037825293347200026), 'episode_return_mean': -280.9065363885516, 'weights_seq_no': 1.0, 'agent_episode_returns_mean': {'pursuer_1': -200.61821661608354, 'pursuer_0': -80.28831977246807}, 'episode_return_max': -245.02516651073142, 'num_module_steps_sampled_lifetime': {'pursuer_1': 4008.0, 'pursuer_0': 4000.0}, 'agent_steps': {'pursuer_1': 500.0, 'pursuer_0': 500.0}, 'env_to_module_sum_episodes_length_out': np.float64(891.0423566894266), 'connector_pipeline_timer': np.float64(0.0002976039977511391), 'rlmodule_inference_timer': np.float64(9.30752296218829e-05), 'episode_len_min': 1000, 'time_between_sampling': np.float64(2.8215732675016625), 'num_env_steps_sampled_lifetime_throughput': np.float64(1123.3565347627766)}","{'num_healthy_workers': 2, 'num_remote_worker_restarts': 0}","{'pursuer_1': {'num_module_steps_trained_lifetime': 120320, 'num_trainable_parameters': 129285, 'weights_seq_no': 2.0, 'vf_loss': np.float32(9.69729), 'num_module_steps_trained': 60160, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.20000000298023224, 'policy_loss': np.float32(-0.00037758052), 'vf_explained_var': np.float32(-0.001319766), 'diff_num_grad_updates_vs_sampler_policy': np.float32(1.0), 'entropy': np.float32(2.853284), 'vf_loss_unclipped': np.float32(956.07336), 'default_optimizer_learning_rate': 5e-05, 'module_train_batch_size_mean': 128.0, 'mean_kl_loss': np.float32(0.007978439), 'gradients_default_optimizer_global_norm': np.float32(1.9866033), 'total_loss': np.fl

...

2025-06-24 15:13:37,141	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/qrbao/ray_results/PPO_2025-06-24_15-13-23' in 0.0112s.
(PPO(env=env; env-runners=2; learners=0; multi-agent=True) pid=552889) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/qrbao/ray_results/PPO_2025-06-24_15-13-23/PPO_env_55877_00000_0_2025-06-24_15-13-23/checkpoint_000000)


== Status ==
Current time: 2025-06-24 15:13:37 (running for 00:00:13.40)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-13-22_768088_547330/artifacts/2025-06-24_15-13-23/PPO_2025-06-24_15-13-23/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+---------------------+------------+---------------------+--------+------------------+------+-------------------+--------------------+--------------------+
| Trial name          | status     | loc                 |   iter |   total time (s) |   ts |   combined return |   return pursuer_0 |   return pursuer_1 |
|---------------------+------------+---------------------+--------+------------------+------+-------------------+--------------------+--------------------|
| PPO_env_55877_00000 | TERMINATED | 192.168.0.25:552889 |      2 |          9.28754 | 8000 |          -280.907 |           -80.2883 |           -200.618 |
+------------------

2025-06-24 15:13:37,634	INFO tune.py:1041 -- Total run time: 13.91 seconds (13.39 seconds for the tuning loop).


环境注册和配置完成


(raylet) [2025-06-24 15:13:43,213 E 551413 551443] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2025-06-24_15-13-22_768088_547330 is over 95% full, available space: 5.49796 GB; capacity: 456.175 GB. Object creation will fail if spilling is required.
(raylet) [2025-06-24 15:13:53,221 E 551413 551443] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2025-06-24_15-13-22_768088_547330 is over 95% full, available space: 5.49795 GB; capacity: 456.175 GB. Object creation will fail if spilling is required.
(raylet) [2025-06-24 15:14:03,229 E 551413 551443] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2025-06-24_15-13-22_768088_547330 is over 95% full, available space: 5.49794 GB; capacity: 456.175 GB. Object creation will fail if spilling is required.
(raylet) [2025-06-24 15:14:13,236 E 551413 551443] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2025-06-24_15-13-22_768088_547330 is over 95% full, available space: 5.49793 GB; capacity: 456.175 GB. Object creation

In [22]:

    # Now swap in the RLModule weights for policy 0.
    chkpt_path = results.get_best_result().checkpoint.path
    p_0_module_state_path = (
        Path(chkpt_path)  # <- algorithm's checkpoint dir
        / COMPONENT_LEARNER_GROUP  # <- learner group
        / COMPONENT_LEARNER  # <- learner
        / COMPONENT_RL_MODULE  # <- MultiRLModule
        / "pursuer_0"  # <- (single) RLModule
    )

    class LoadP0OnAlgoInitCallback(DefaultCallbacks):
        def on_algorithm_init(self, *, algorithm, **kwargs):
            module_p0 = algorithm.get_module("pursuer_0")
            weight_before = convert_to_numpy(next(iter(module_p0.parameters())))
            algorithm.restore_from_path(
                p_0_module_state_path,
                component=(
                    COMPONENT_LEARNER_GROUP
                    + "/"
                    + COMPONENT_LEARNER
                    + "/"
                    + COMPONENT_RL_MODULE
                    + "/pursuer_0"
                ),
            )
            # Make sure weights were updated.
            weight_after = convert_to_numpy(next(iter(module_p0.parameters())))
            check(weight_before, weight_after, false=True)

    base_config.callbacks(LoadP0OnAlgoInitCallback)

    # Define stopping criteria.
    stop = {
        f"{ENV_RUNNER_RESULTS}/{EPISODE_RETURN_MEAN}": -800.0,
        f"{ENV_RUNNER_RESULTS}/{NUM_ENV_STEPS_SAMPLED_LIFETIME}": 100000,
        TRAINING_ITERATION: 100,
    }

    # Run the experiment again with the restored MultiRLModule.



In [23]:
run_rllib_example_script_experiment(base_config, args, stop=stop)

2025-06-24 15:14:21,969	INFO worker.py:1747 -- Calling ray.init() again after it has already been called.


== Status ==
Current time: 2025-06-24 15:14:22 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-13-22_768088_547330/artifacts/2025-06-24_15-14-21/PPO_2025-06-24_15-14-21/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-06-24 15:14:27 (running for 00:00:05.16)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-13-22_768088_547330/artifacts/2025-06-24_15-14-21/PPO_2025-06-24_15-14-21/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




Trial name,env_runner_group,env_runners,fault_tolerance,learners,num_env_steps_sampled_lifetime,num_training_step_calls_per_iteration,perf,timers
PPO_env_783e9_00000,{'actor_manager_num_outstanding_async_reqs': 0},"{'episode_return_mean': -268.5760928226974, 'timers': {'connectors': {'add_states_from_episodes_to_batch': np.float64(5.258996679913253e-06), 'add_time_dim_to_batch_and_zero_pad': np.float64(1.3854005374014378e-05), 'batch_individual_items': np.float64(2.0836996554862708e-05), 'add_observations_from_episodes_to_batch': np.float64(2.7727000997401774e-05), 'numpy_to_tensor': np.float64(4.313700628699735e-05), 'agent_to_module_mapping': np.float64(4.517001798376441e-06)}}, 'env_step_timer': np.float64(0.0003814526646025043), 'module_to_env_connector': {'timers': {'connectors': {'listify_data_for_vector_env': np.float64(5.942108929210167e-06), 'get_actions': np.float64(8.389884802789747e-05), 'remove_single_ts_time_rank_from_batch': np.float64(1.0195315071435456e-06), 'un_batch_to_individual_items': np.float64(1.3324286282875254e-05), 'module_to_agent_unmapping': np.float64(2.590691764361659e-06), 'tensor_to_numpy': np.float64(3.194252529286243e-05), 'normalize_and_clip_actions': np.float64(3.5813801194417956e-05)}}, 'connector_pipeline_timer': np.float64(0.00022830546882528512)}, 'connector_pipeline_timer': np.float64(0.00030506049370160326), 'rlmodule_inference_timer': np.float64(9.454647176123216e-05), 'num_episodes': 4.0, 'num_episodes_lifetime': 4.0, 'episode_duration_sec_mean': 0.9844869287517213, 'env_to_module_sum_episodes_length_out': np.float64(891.0423550617884), 'env_to_module_connector': {'timers': {'connectors': {'numpy_to_tensor': np.float64(2.298203175429458e-05), 'agent_to_module_mapping': np.float64(3.51070875931226e-06), 'add_states_from_episodes_to_batch': np.float64(4.080696212233653e-06), 'add_time_dim_to_batch_and_zero_pad': np.float64(7.276576194437507e-06), 'batch_individual_items': np.float64(1.2931267538449835e-05), 'add_observations_from_episodes_to_batch': np.float64(1.5947523682306034e-05)}}, 'connector_pipeline_timer': np.float64(0.00010796216440645115)}, 'num_module_steps_sampled': {'pursuer_0': 2000.0, 'pursuer_1': 2004.0}, 'num_agent_steps_sampled': {'pursuer_0': 2000.0, 'pursuer_1': 2004.0}, 'num_env_steps_sampled_lifetime': 4000.0, 'episode_return_max': -242.96889415168536, 'episode_len_max': 1000, 'weights_seq_no': 0.0, 'env_reset_timer': np.float64(0.0015530124946963042), 'num_agent_steps_sampled_lifetime': {'pursuer_0': 2000.0, 'pursuer_1': 2004.0}, 'env_to_module_sum_episodes_length_in': np.float64(891.0423550617884), 'agent_steps': {'pursuer_0': 500.0, 'pursuer_1': 500.0}, 'agent_episode_returns_mean': {'pursuer_0': -95.10862909562223, 'pursuer_1': -173.4674637270751}, 'episode_len_mean': 1000.0, 'module_episode_returns_mean': {'pursuer_0': -95.10862909562223, 'pursuer_1': -173.4674637270751}, 'sample': np.float64(1.9823065269956714), 'num_module_steps_sampled_lifetime': {'pursuer_0': 2000.0, 'pursuer_1': 2004.0}, 'episode_len_min': 1000, 'num_env_steps_sampled': 4000.0, 'episode_return_min': -298.266061570744, 'num_env_steps_sampled_lifetime_throughput': np.float64(1120.1655487619191)}","{'num_healthy_workers': 2, 'num_remote_worker_restarts': 0}","{'__all_modules__': {'learner_connector': {'timers': {'connectors': {'add_observations_from_episodes_to_batch': 6.593498983420432e-05, 'add_one_ts_to_episodes_and_truncate': 0.0029576039960375056, 'numpy_to_tensor': 0.0010344220063416287, 'add_time_dim_to_batch_and_zero_pad': 1.7400991055183113e-05, 'batch_individual_items': 0.020714743004646152, 'add_states_from_episodes_to_batch': 5.59899490326643e-06, 'agent_to_module_mapping': 0.001833588001318276, 'add_columns_from_episodes_to_train_batch': 0.034032218987704255, 'general_advantage_estimation': 0.053947927997796796}}, 'connector_pipeline_timer': 0.11501642699295189}, 'learner_connector_sum_episodes_length_in': 4000, 'num_module_steps_trained_lifetime': 120320, 'num_non_trainable_p

2025-06-24 15:14:30,777	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/qrbao/ray_results/PPO_2025-06-24_15-14-21' in 0.0092s.
2025-06-24 15:14:30,860	INFO tune.py:1041 -- Total run time: 8.89 seconds (8.79 seconds for the tuning loop).


== Status ==
Current time: 2025-06-24 15:14:30 (running for 00:00:08.80)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-13-22_768088_547330/artifacts/2025-06-24_15-14-21/PPO_2025-06-24_15-14-21/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+---------------------+------------+---------------------+--------+------------------+------+-------------------+--------------------+--------------------+
| Trial name          | status     | loc                 |   iter |   total time (s) |   ts |   combined return |   return pursuer_0 |   return pursuer_1 |
|---------------------+------------+---------------------+--------+------------------+------+-------------------+--------------------+--------------------|
| PPO_env_783e9_00000 | TERMINATED | 192.168.0.25:553224 |      1 |          4.80809 | 4000 |          -268.576 |           -95.1086 |           -173.467 |
+------------------

ResultGrid<[
  Result(
    metrics={'timers': {'training_iteration': 4.805108229003963, 'restore_env_runners': 1.4025994460098445e-05, 'training_step': 4.80497099198692, 'env_runner_sampling_timer': 1.9941081529977964, 'learner_update_timer': 2.8083153249899624, 'synch_weights': 0.00232674898870755}, 'env_runners': {'episode_return_mean': -268.5760928226974, 'timers': {'connectors': {'add_states_from_episodes_to_batch': np.float64(5.258996679913253e-06), 'add_time_dim_to_batch_and_zero_pad': np.float64(1.3854005374014378e-05), 'batch_individual_items': np.float64(2.0836996554862708e-05), 'add_observations_from_episodes_to_batch': np.float64(2.7727000997401774e-05), 'numpy_to_tensor': np.float64(4.313700628699735e-05), 'agent_to_module_mapping': np.float64(4.517001798376441e-06)}}, 'env_step_timer': np.float64(0.0003814526646025043), 'module_to_env_connector': {'timers': {'connectors': {'listify_data_for_vector_env': np.float64(5.942108929210167e-06), 'get_actions': np.float64(8.3898848